In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def read_depth(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".npy":
        arr = np.load(path).astype(np.float32)
    else:
        arr = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        arr = arr.astype(np.float32)
    return arr

def read_gray_or_color(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def color_to_gray_depth(pred_img_colored):
    if pred_img_colored.ndim == 3:
        print('yes')
        gray = cv2.cvtColor(pred_img_colored, cv2.COLOR_RGB2GRAY).astype(np.float32) / 255.0
    else:
        gray = pred_img_colored.astype(np.float32)
        if gray.max() > 1.5:
            gray = gray / 255.0
    return gray



def read_depth(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".npy":
        arr = np.load(path).astype(np.float32)
    else:
        arr = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        arr = arr.astype(np.float32)
    return arr

def read_gray_or_color(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def color_to_gray_depth(pred_img_colored):
    if pred_img_colored.ndim == 3:
        print('yes')
        gray = cv2.cvtColor(pred_img_colored, cv2.COLOR_RGB2GRAY).astype(np.float32) / 255.0
    else:
        gray = pred_img_colored.astype(np.float32)
        if gray.max() > 1.5:
            gray = gray / 255.0
    return gray

def stem(p):
    return os.path.splitext(os.path.basename(p))[0]

def align_scale_shift(pred, gt, valid_mask=None):
    """Find scale and shift to best fit pred to gt using least squares."""
    pred_f = pred.astype(np.float32)
    gt_f = gt.astype(np.float32)
    if valid_mask is None:
        valid_mask = np.ones_like(pred, dtype=bool)
    A = np.vstack([pred_f[valid_mask], np.ones(np.sum(valid_mask))]).T
    scale, shift = np.linalg.lstsq(A, gt_f[valid_mask], rcond=None)[0]
    return scale, shift

def remove_outliers(pred, ksize=3, threshold=20):
    """Remove outliers by clipping pixels far from neighborhood median."""
    med = cv2.medianBlur(pred, ksize=ksize)
    diff = np.abs(pred.astype(np.float32) - med.astype(np.float32))
    pred_clean = pred.copy()
    pred_clean[diff > threshold] = med[diff > threshold]
    return pred_clean

def depth_metrics(pred, gt, valid_mask, eps=1e-6):
    p = pred[valid_mask].astype(np.float32)
    g = gt[valid_mask].astype(np.float32)

    rmse = np.sqrt(np.mean((p - g)**2))
    mae  = np.mean(np.abs(p - g))
    absrel = np.mean(np.abs(p - g) / np.maximum(g, eps))
    sqrel  = np.mean(((p - g)**2) / np.maximum(g, eps))

    ratio = np.maximum(p / np.maximum(g, eps), g / np.maximum(p, eps))
    d1   = np.mean(ratio < 1.25)
    d2   = np.mean(ratio < 1.25**2)
    d3   = np.mean(ratio < 1.25**3)

    return dict(RMSE=rmse, MAE=mae, AbsRel=absrel, SqRel=sqrel,
                delta_1=d1, delta_2=d2, delta_3=d3)

def disparity_metrics(pred_disp, gt_disp, valid_mask):
    p = pred_disp[valid_mask].astype(np.float32)
    g = gt_disp[valid_mask].astype(np.float32)

    epe = np.mean(np.abs(p - g))
    bad = np.logical_and(np.abs(p - g) > 3.0, np.abs(p - g) / np.maximum(np.abs(g), 1e-6) > 0.05)
    bad3 = np.mean(bad)

    return dict(EPE=epe, Bad3=bad3)




In [ ]:
# final
import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

gt_dir = "/content/drive/MyDrive/data/images/gt_depth"
pred_dir = "/content/drive/MyDrive/output_depth_DepthPro"
rgb_dir = "/content/drive/MyDrive/data/images/left"
output_csv = "depth_quantized_metrics_calibrated_smooth.csv"
out_fig_dir = "qualitative_figs"
os.makedirs(out_fig_dir, exist_ok=True)

def stem(p):
    return os.path.splitext(os.path.basename(p))[0]

def align_scale_shift(pred, gt, valid_mask=None):
    """Find scale and shift to best fit pred to gt using least squares."""
    pred_f = pred.astype(np.float32)
    gt_f = gt.astype(np.float32)
    if valid_mask is None:
        valid_mask = np.ones_like(pred, dtype=bool)
    A = np.vstack([pred_f[valid_mask], np.ones(np.sum(valid_mask))]).T
    scale, shift = np.linalg.lstsq(A, gt_f[valid_mask], rcond=None)[0]
    return scale, shift

def remove_outliers(pred, ksize=3, threshold=20):
    """Remove outliers by clipping pixels far from neighborhood median."""
    med = cv2.medianBlur(pred, ksize=ksize)
    diff = np.abs(pred.astype(np.float32) - med.astype(np.float32))
    pred_clean = pred.copy()
    pred_clean[diff > threshold] = med[diff > threshold]
    return pred_clean

def save_qualitative_figure(name, rgb, gt_depth, pred_depth_aligned, valid_mask):
    #pred_depth_aligned = 1.0 - pred_depth_aligned
    abs_err = np.zeros_like(gt_depth, dtype=np.float32)
    abs_err[valid_mask] = np.abs(pred_depth_aligned[valid_mask] - gt_depth[valid_mask])

    def viz_depth(d):
        m = np.percentile(d[valid_mask], 2)
        M = np.percentile(d[valid_mask], 98)
        d2 = np.clip((d - m) / max(M - m, 1e-6), 0, 1)
        return (plt.cm.inferno(d2)[:, :, :3] * 255).astype(np.uint8)

    gt_viz   = viz_depth(gt_depth)
    pred_viz = viz_depth(pred_depth_aligned)

    err_norm = abs_err.copy()
    if np.any(valid_mask):
        m = np.percentile(abs_err[valid_mask], 2)
        M = np.percentile(abs_err[valid_mask], 98)
        err_norm = np.clip((abs_err - m) / max(M - m, 1e-6), 0, 1)
    err_viz = (plt.cm.viridis(err_norm)[:, :, :3] * 255).astype(np.uint8)

    tiles = []
    if rgb is not None:
        tiles.append(rgb)
    tiles.extend([gt_viz, pred_viz, err_viz])

    h = min(t.shape[0] for t in tiles)
    tiles = [cv2.resize(t, (int(t.shape[1]*h/t.shape[0]), h)) for t in tiles]
    concat = np.concatenate(tiles, axis=1)

    out_path = os.path.join(out_fig_dir, f"{name}_qual.png")
    cv2.imwrite(out_path, cv2.cvtColor(concat, cv2.COLOR_RGB2BGR))
    return out_path

gt_paths = {stem(p): p for p in glob.glob(os.path.join(gt_dir, "*.png"))}
pred_paths = {stem(p): p for p in glob.glob(os.path.join(pred_dir, "*.png"))}

common_keys = sorted(set(gt_paths.keys()) & set(pred_paths.keys()))
print(f"Found {len(common_keys)} common files.")
def delta_metrics(pred, gt, valid_mask):
    pred = pred[valid_mask]
    gt = gt[valid_mask]
    ratio = np.maximum(pred / (gt + 1e-6), gt / (pred + 1e-6))
    d1 = np.mean(ratio < 1.25)
    d2 = np.mean(ratio < 1.25**2)
    d3 = np.mean(ratio < 1.25**3)
    return d1, d2, d3

rows = []

for k in tqdm(common_keys):
    gt = cv2.imread(gt_paths[k], cv2.IMREAD_UNCHANGED).astype(np.float32)
    pred = cv2.imread(pred_paths[k], cv2.IMREAD_UNCHANGED).astype(np.float32)

    if gt.shape != pred.shape:
        pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_NEAREST)


    pred_inverted = 255.0 - pred

    pred_norm = (pred_inverted - pred_inverted.min()) / (pred_inverted.max() - pred_inverted.min()) * (gt.max() - gt.min()) + gt.min()

    # smoothing: bilateral + Gaussian
    pred_smooth = cv2.bilateralFilter(pred_norm.astype(np.float32), d=9, sigmaColor=75, sigmaSpace=75)
    pred_smooth = cv2.GaussianBlur(pred_smooth, (5,5), sigmaX=1.0)

    # outlier removal
    pred_smooth = remove_outliers(pred_smooth, ksize=3, threshold=15)

    # calibration: scale + shift
    valid_mask = gt > 0
    scale, shift = align_scale_shift(pred_smooth, gt, valid_mask)
    pred_aligned = np.clip(scale * pred_smooth + shift, gt.min(), gt.max())

    #  metrics
    rmse = np.sqrt(np.mean((pred_aligned - gt)**2))
    mae  = np.mean(np.abs(pred_aligned - gt))
    absrel = np.mean(np.abs(pred_aligned - gt) / (gt + 1e-6))
    d1, d2, d3 = delta_metrics(pred_aligned, gt, valid_mask)

    rows.append({
        "name": k,
        "RMSE": rmse,
        "MAE": mae,
        "AbsRel": absrel,
        "δ<1.25": d1,
        "δ<1.25²": d2,
        "δ<1.25³": d3,
        "GT_min": gt.min(),
        "GT_max": gt.max(),
        "scale": scale,
        "shift": shift
    })


    rgb = None
    rgb_path = os.path.join(rgb_dir, k + ".png")
    if os.path.exists(rgb_path):
        rgb = cv2.imread(rgb_path)[:, :, ::-1]  # BGR -> RGB
    save_qualitative_figure(k, rgb, gt, pred_aligned, valid_mask)

df = pd.DataFrame(rows)
df.sort_values("name", inplace=True)
df.to_csv(output_csv, index=False)
print(f"Saved quantized metrics with full calibration + smoothing to {output_csv}")

display(df.describe().transpose())

best5 = df.nsmallest(5, "RMSE")
worst5 = df.nlargest(5, "RMSE")

os.makedirs("best_cases", exist_ok=True)
os.makedirs("worst_cases", exist_ok=True)

for k in best5["name"]:
    src = os.path.join(out_fig_dir, f"{k}_qual.png")
    dst = os.path.join("best_cases", f"{k}_qual.png")
    if os.path.exists(src):
        os.system(f"cp '{src}' '{dst}'")

for k in worst5["name"]:
    src = os.path.join(out_fig_dir, f"{k}_qual.png")
    dst = os.path.join("worst_cases", f"{k}_qual.png")
    if os.path.exists(src):
        os.system(f"cp '{src}' '{dst}'")

print("Saved Top-5 best cases in 'best_cases/' and worst cases in 'worst_cases/'")




Found 50 common files.


100%|██████████| 50/50 [01:42<00:00,  2.06s/it]

Saved quantized metrics with full calibration + smoothing to depth_quantized_metrics_calibrated_smooth.csv


,count,mean,std,min,25%,50%,75%,max
RMSE,50.0,36.491188,8.810281,23.886987,29.686585,35.081738,41.985061,59.048509
MAE,50.0,19.651785,7.437727,9.049567,14.625627,18.484218,24.353735,37.838866
AbsRel,50.0,2.125050,1.407796,0.271579,0.688112,2.376027,3.331734,4.820906
δ<1.25,50.0,0.263139,0.096406,0.101025,0.185049,0.238434,0.362008,0.493173
δ<1.25²,50.0,0.464925,0.145452,0.250981,0.360844,0.430184,0.512615,0.900112
δ<1.25³,50.0,0.573444,0.171572,0.314447,0.435895,0.529497,0.673919,0.957658
GT_min,50.0,2.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000
GT_max,50.0,255.000000,0.000000,255.000000,255.000000,255.000000,255.000000,255.000000
scale,50.0,-13.635394,22.735072,-140.989215,-20.823032,-4.773308,-1.644450,-0.983345
shift,50.0,3493.391452,5799.491735,279.393229,421.503814,1220.529251,5332.522858,35978.450303


Saved Top-5 best cases in 'best_cases/' and worst cases in 'worst_cases/'


In [ ]:
best = df.nsmallest(1, "RMSE")
worst = df.nlargest(1, "RMSE")


In [ ]:
best

,name,RMSE,MAE,AbsRel,δ<1.25,δ<1.25²,δ<1.25³,GT_min,GT_max,scale,shift
29,495,23.886987,9.136143,0.734161,0.38347,0.502658,0.638126,2.0,255.0,-1.605902,415.85991


In [ ]:
worst

,name,RMSE,MAE,AbsRel,δ<1.25,δ<1.25²,δ<1.25³,GT_min,GT_max,scale,shift
26,492,59.048509,37.785433,4.716508,0.111627,0.265199,0.327579,2.0,255.0,-1.285651,363.804581


In [ ]:
import os
import sys

# طريقة لإعادة تشغيل الـ runtime
def restart_runtime():
    print("Restarting runtime...")
    os._exit(00)  # بيخرج من الـ process الحالي → Colab بيعمل restart تلقائي


restart_runtime()

In [ ]:


import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

gt_dir = "/content/drive/MyDrive/data/images/gt_depth"
pred_dir = "/content/drive/MyDrive/pre_depth"
rgb_dir = "/content/drive/MyDrive/data/images/left"   # اضفته علشان صور RGB
output_csv = "depth_quantized_metrics_calibrated_smooth.csv"
out_fig_dir = "qualitative_figs"  #統一 مع save_qualitative_figure
os.makedirs(out_fig_dir, exist_ok=True)

def stem(p):
    return os.path.splitext(os.path.basename(p))[0]

def align_scale_shift(pred, gt, valid_mask=None):
    """Find scale and shift to best fit pred to gt using least squares."""
    pred_f = pred.astype(np.float32)
    gt_f = gt.astype(np.float32)
    if valid_mask is None:
        valid_mask = np.ones_like(pred, dtype=bool)
    A = np.vstack([pred_f[valid_mask], np.ones(np.sum(valid_mask))]).T
    scale, shift = np.linalg.lstsq(A, gt_f[valid_mask], rcond=None)[0]
    return scale, shift

def remove_outliers(pred, ksize=3, threshold=20):
    """Remove outliers by clipping pixels far from neighborhood median."""
    med = cv2.medianBlur(pred, ksize=ksize)
    diff = np.abs(pred.astype(np.float32) - med.astype(np.float32))
    pred_clean = pred.copy()
    pred_clean[diff > threshold] = med[diff > threshold]
    return pred_clean

# موجودة عندك من قبل
def save_qualitative_figure(name, rgb, gt_depth, pred_depth_aligned, valid_mask):
    #pred_depth_aligned = 1.0 - pred_depth_aligned
    abs_err = np.zeros_like(gt_depth, dtype=np.float32)
    abs_err[valid_mask] = np.abs(pred_depth_aligned[valid_mask] - gt_depth[valid_mask])

    def viz_depth(d):
        m = np.percentile(d[valid_mask], 2)
        M = np.percentile(d[valid_mask], 98)
        d2 = np.clip((d - m) / max(M - m, 1e-6), 0, 1)
        return (plt.cm.inferno(d2)[:, :, :3] * 255).astype(np.uint8)

    gt_viz   = viz_depth(gt_depth)
    pred_viz = viz_depth(pred_depth_aligned)

    err_norm = abs_err.copy()
    if np.any(valid_mask):
        m = np.percentile(abs_err[valid_mask], 2)
        M = np.percentile(abs_err[valid_mask], 98)
        err_norm = np.clip((abs_err - m) / max(M - m, 1e-6), 0, 1)
    err_viz = (plt.cm.viridis(err_norm)[:, :, :3] * 255).astype(np.uint8)

    tiles = []
    if rgb is not None:
        tiles.append(rgb)
    tiles.extend([gt_viz, pred_viz, err_viz])

    h = min(t.shape[0] for t in tiles)
    tiles = [cv2.resize(t, (int(t.shape[1]*h/t.shape[0]), h)) for t in tiles]
    concat = np.concatenate(tiles, axis=1)

    out_path = os.path.join(out_fig_dir, f"{name}_qual.png")
    cv2.imwrite(out_path, cv2.cvtColor(concat, cv2.COLOR_RGB2BGR))
    return out_path

gt_paths = {stem(p): p for p in glob.glob(os.path.join(gt_dir, "*.png"))}
pred_paths = {stem(p): p for p in glob.glob(os.path.join(pred_dir, "*.png"))}

common_keys = sorted(set(gt_paths.keys()) & set(pred_paths.keys()))
print(f"Found {len(common_keys)} common files.")
# --- إضافة δ-metrics ---
def delta_metrics(pred, gt, valid_mask):
    pred = pred[valid_mask]
    gt = gt[valid_mask]
    ratio = np.maximum(pred / (gt + 1e-6), gt / (pred + 1e-6))
    d1 = np.mean(ratio < 1.25)
    d2 = np.mean(ratio < 1.25**2)
    d3 = np.mean(ratio < 1.25**3)
    return d1, d2, d3

rows = []

for k in tqdm(common_keys):
    gt = cv2.imread(gt_paths[k], cv2.IMREAD_UNCHANGED).astype(np.float32)
    pred = cv2.imread(pred_paths[k], cv2.IMREAD_UNCHANGED).astype(np.float32)

    # resize لو مختلف
    if gt.shape != pred.shape:
        pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_NEAREST)

    # عكس prediction عشان الأبيض = بعيد زي GT
    pred_inverted = 255.0 - pred

    # normalization للـ prediction
    pred_norm = (pred_inverted - pred_inverted.min()) / (pred_inverted.max() - pred_inverted.min()) * (gt.max() - gt.min()) + gt.min()

    # smoothing: bilateral + Gaussian
    pred_smooth = cv2.bilateralFilter(pred_norm.astype(np.float32), d=9, sigmaColor=75, sigmaSpace=75)
    pred_smooth = cv2.GaussianBlur(pred_smooth, (5,5), sigmaX=1.0)

    # outlier removal
    pred_smooth = remove_outliers(pred_smooth, ksize=3, threshold=15)

    # calibration: scale + shift
    valid_mask = gt > 0
    scale, shift = align_scale_shift(pred_smooth, gt, valid_mask)
    pred_aligned = np.clip(scale * pred_smooth + shift, gt.min(), gt.max())

    # حساب metrics
    rmse = np.sqrt(np.mean((pred_aligned - gt)**2))
    mae  = np.mean(np.abs(pred_aligned - gt))
    absrel = np.mean(np.abs(pred_aligned - gt) / (gt + 1e-6))
    d1, d2, d3 = delta_metrics(pred_aligned, gt, valid_mask)

    rows.append({
        "name": k,
        "RMSE": rmse,
        "MAE": mae,
        "AbsRel": absrel,
        "δ<1.25": d1,
        "δ<1.25²": d2,
        "δ<1.25³": d3,
        "GT_min": gt.min(),
        "GT_max": gt.max(),
        "scale": scale,
        "shift": shift
    })

    # حفظ صور المقارنة
    rgb = None
    rgb_path = os.path.join(rgb_dir, k + ".png")
    if os.path.exists(rgb_path):
        rgb = cv2.imread(rgb_path)[:, :, ::-1]  # BGR -> RGB
    save_qualitative_figure(k, rgb, gt, pred_aligned, valid_mask)

df = pd.DataFrame(rows)
df.sort_values("name", inplace=True)
df.to_csv(output_csv, index=False)
print(f"Saved quantized metrics with full calibration + smoothing to {output_csv}")

display(df.describe().transpose())

# --- أفضل وأسوأ 5 ---
best5 = df.nsmallest(5, "RMSE")
worst5 = df.nlargest(5, "RMSE")

os.makedirs("best_cases", exist_ok=True)
os.makedirs("worst_cases", exist_ok=True)

for k in best5["name"].astype(str):
    src = os.path.join(out_fig_dir, f"{k}_qual.png")
    print("best",src)
    dst = os.path.join("best_cases", f"{k}_qual.png")
    if os.path.exists(src):
        import shutil
        shutil.copy(src, dst)


for k in worst5["name"].astype(str):
    src = os.path.join(out_fig_dir, f"{k}_qual.png")
    print("worst",src)
    dst = os.path.join("worst_cases", f"{k}_qual.png")
    if os.path.exists(src):
        shutil.copy(src, dst)


print("Saved Top-5 best cases in 'best_cases/' and worst cases in 'worst_cases/'")




Found 50 common files.


100%|██████████| 50/50 [01:19<00:00,  1.58s/it]

Saved quantized metrics with full calibration + smoothing to depth_quantized_metrics_calibrated_smooth.csv


,count,mean,std,min,25%,50%,75%,max
RMSE,50.0,28.264942,3.014370,23.561943,25.845477,27.522019,30.289820,34.202215
MAE,50.0,17.741939,1.722098,14.452322,16.527280,17.383829,18.703967,21.791568
AbsRel,50.0,2.174801,0.309483,1.567971,1.928392,2.127757,2.386657,2.920896
δ<1.25,50.0,0.197397,0.031864,0.149404,0.173083,0.189836,0.220093,0.278322
δ<1.25²,50.0,0.324708,0.048683,0.252566,0.281583,0.304927,0.370461,0.398598
δ<1.25³,50.0,0.455381,0.034546,0.379098,0.433814,0.463024,0.484524,0.502856
GT_min,50.0,2.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000
GT_max,50.0,255.000000,0.000000,255.000000,255.000000,255.000000,255.000000,255.000000
scale,50.0,0.882469,0.041972,0.789538,0.864286,0.891714,0.908156,0.940722
shift,50.0,16.131360,2.287569,11.403185,14.462797,16.027742,17.227950,21.417061


best qualitative_figs/491_qual.png
best qualitative_figs/490_qual.png
best qualitative_figs/489_qual.png
best qualitative_figs/492_qual.png
best qualitative_figs/483_qual.png
worst qualitative_figs/505_qual.png
worst qualitative_figs/500_qual.png
worst qualitative_figs/506_qual.png
worst qualitative_figs/502_qual.png
worst qualitative_figs/508_qual.png
Saved Top-5 best cases in 'best_cases/' and worst cases in 'worst_cases/'
